In [1]:
import pandas as pd

# Define file paths
mc_co_file_path = '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/mc/investment_results_final(2006)(train_1years_c-c).xlsx'
mc_oc_file_path = '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/mc/investment_results_final(2006)(train_1years_c-c).xlsx'
mc_cc_file_path = '/usr3/graduate/xz0224/normal/mc/investment_results_final(2006)(train_1years_c-c).xlsx'
qq_co_file_path = '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/investment_results_final(2006)(train_3month_c-c).xlsx'
qq_cc_file_path = '/usr3/graduate/xz0224/normal/qq/investment_results_final(2006)(train_3month_c-c).xlsx'
qq_3m_co_file_path = '/usr3/graduate/xz0224/co（就是从close买，明天的open卖）/qq/qq 3月很高.xlsx'
mc_1y_oc_file_path = '/usr3/graduate/xz0224/oc（就是从今天的open买，今天的close卖）/mc/investment_results_final(2006)(train_1years_c-c).xlsx'

# Load data
mc_co_data = pd.read_excel(mc_co_file_path)
mc_oc_data = pd.read_excel(mc_oc_file_path)
mc_cc_data = pd.read_excel(mc_cc_file_path)
qq_co_data = pd.read_excel(qq_co_file_path)
qq_cc_data = pd.read_excel(qq_cc_file_path)
qq_3m_co_data = pd.read_excel(qq_3m_co_file_path)
mc_1y_oc_data = pd.read_excel(mc_1y_oc_file_path)

# Merge strategies
merged_m1y_q3m_data = pd.merge(mc_1y_oc_data, qq_3m_co_data, on='Date', suffixes=('_oc', '_co'))

# Initialize variables to keep track of the current state and transaction count for the merged strategy
current_state = 'cash'
transaction_count = 0
transaction_log = []

# Simulate the combined strategy
for index, row in merged_m1y_q3m_data.iterrows():
    date = row['Date']
    decision_oc = row['Decision_oc']
    decision_co = row['Decision_co']

    # Perform the oc transaction
    if current_state != decision_oc:
        transaction_count += 1
        transaction_log.append((date, 'oc', current_state, decision_oc))
        current_state = decision_oc

    # Perform the co transaction
    if current_state != decision_co:
        transaction_count += 1
        transaction_log.append((date, 'co', current_state, decision_co))
        current_state = decision_co

# Create a DataFrame to display the transaction log
merged_m1y_q3m_transaction_log_df = pd.DataFrame(transaction_log, columns=['Date', 'Strategy', 'From', 'To'])

# Function to calculate transaction count for individual strategies
def calculate_transaction_count(data):
    current_state = 'cash'
    transaction_count = 0
    transaction_log = []

    for index, row in data.iterrows():
        date = row['Date']
        decision = row['Decision']

        if current_state != decision:
            transaction_count += 1
            transaction_log.append((date, current_state, decision))
            current_state = decision

    return pd.DataFrame(transaction_log, columns=['Date', 'From', 'To'])

# Calculate transaction cost for other strategies
mc_cc_transaction_log_df = calculate_transaction_count(mc_cc_data)
qq_cc_transaction_log_df = calculate_transaction_count(qq_cc_data)
qq_co_transaction_log_df = calculate_transaction_count(qq_co_data)

# Function to calculate annual transaction count
def calculate_annual_transaction_count(transaction_log_df, start_year, end_year):
    transaction_log_df['Year'] = pd.to_datetime(transaction_log_df['Date']).dt.year
    annual_transaction_count = transaction_log_df.groupby('Year').size().reindex(range(start_year, end_year+1), fill_value=0).reset_index(name='Transaction_Count')
    return annual_transaction_count

start_year = min(
    mc_cc_transaction_log_df['Date'].dt.year.min(),
    qq_cc_transaction_log_df['Date'].dt.year.min(),
    qq_co_transaction_log_df['Date'].dt.year.min(),
    merged_m1y_q3m_transaction_log_df['Date'].dt.year.min()
)

end_year = max(
    mc_cc_transaction_log_df['Date'].dt.year.max(),
    qq_cc_transaction_log_df['Date'].dt.year.max(),
    qq_co_transaction_log_df['Date'].dt.year.max(),
    merged_m1y_q3m_transaction_log_df['Date'].dt.year.max()
)

merged_m1y_q3m_annual_transaction_count = calculate_annual_transaction_count(merged_m1y_q3m_transaction_log_df, start_year, end_year)
mc_cc_annual_transaction_count = calculate_annual_transaction_count(mc_cc_transaction_log_df, start_year, end_year)
qq_cc_annual_transaction_count = calculate_annual_transaction_count(qq_cc_transaction_log_df, start_year, end_year)
qq_co_annual_transaction_count = calculate_annual_transaction_count(qq_co_transaction_log_df, start_year, end_year)

# Add statistical analysis
def add_statistical_analysis(df):
    max_values = df.iloc[:, 1:].max()
    min_values = df.iloc[:, 1:].min()
    median_values = df.iloc[:, 1:].median()
    average_values = df.iloc[:, 1:].mean()
    stddev_values = df.iloc[:, 1:].std()
    return pd.DataFrame({
        'Max': max_values,
        'Min': min_values,
        'Median': median_values,
        'Mean': average_values,
        'StdDev': stddev_values
    })

merged_m1y_q3m_stats = add_statistical_analysis(merged_m1y_q3m_annual_transaction_count)
mc_cc_stats = add_statistical_analysis(mc_cc_annual_transaction_count)
qq_cc_stats = add_statistical_analysis(qq_cc_annual_transaction_count)
qq_co_stats = add_statistical_analysis(qq_co_annual_transaction_count)

# Combine annual transaction count and statistical analysis into a single DataFrame
def combine_annual_and_stats(annual_df, stats_df, strategy_name):
    combined_df = annual_df.copy()
    stats_df.reset_index(inplace=True)
    combined_df = pd.concat([combined_df, stats_df], axis=1)
    combined_df['Strategy'] = strategy_name
    return combined_df

merged_m1y_q3m_combined = combine_annual_and_stats(merged_m1y_q3m_annual_transaction_count, merged_m1y_q3m_stats, 'Merged_M1Y_Q3M')
mc_cc_combined = combine_annual_and_stats(mc_cc_annual_transaction_count, mc_cc_stats, 'MC_CC')
qq_cc_combined = combine_annual_and_stats(qq_cc_annual_transaction_count, qq_cc_stats, 'QQ_CC')
qq_co_combined = combine_annual_and_stats(qq_co_annual_transaction_count, qq_co_stats, 'QQ_CO')

# Prepare the final DataFrame
final_df = pd.DataFrame()
final_df['Year'] = merged_m1y_q3m_annual_transaction_count['Year']
final_df['Merged_M1Y_Q3M'] = merged_m1y_q3m_annual_transaction_count['Transaction_Count']
final_df['MC_CC'] = mc_cc_annual_transaction_count['Transaction_Count']
final_df['QQ_CC'] = qq_cc_annual_transaction_count['Transaction_Count']
final_df['QQ_CO'] = qq_co_annual_transaction_count['Transaction_Count']

# Append statistics
stats_row = pd.DataFrame({
    'Year': ['$\\max$', '$\\min$', '$M$', '$\\mu$', '$\\sigma$'],
    'Merged_M1Y_Q3M': [merged_m1y_q3m_stats['Max'][0], merged_m1y_q3m_stats['Min'][0], merged_m1y_q3m_stats['Median'][0], merged_m1y_q3m_stats['Mean'][0], merged_m1y_q3m_stats['StdDev'][0]],
    'MC_CC': [mc_cc_stats['Max'][0], mc_cc_stats['Min'][0], mc_cc_stats['Median'][0], mc_cc_stats['Mean'][0], mc_cc_stats['StdDev'][0]],
    'QQ_CC': [qq_cc_stats['Max'][0], qq_cc_stats['Min'][0], qq_cc_stats['Median'][0], qq_cc_stats['Mean'][0], qq_cc_stats['StdDev'][0]],
    'QQ_CO': [qq_co_stats['Max'][0], qq_co_stats['Min'][0], qq_co_stats['Median'][0], qq_co_stats['Mean'][0], qq_co_stats['StdDev'][0]]
})

final_df = pd.concat([final_df, stats_row], ignore_index=True)

# Save the combined results to a single CSV file
final_df.to_csv('/usr3/graduate/xz0224/all_strategies_transaction_analysis.csv', index=False)
